In [ ]:
from transformers.cache_utils import Cache, DynamicCache, StaticCache
import torch, time
from transformers import AutoConfig
import sys
from typing import Optional, Union
sys.path.insert(0, '/mnt/zhengcf3/lpllm')
from lpllm.lpllm import LPLLM
from lpllm.StaticCacheLen import StaticCacheLen

config = AutoConfig.from_pretrained(
            f"/mnt/zhengcf3/models/deepseek-moe-16b-base", trust_remote_code=True
        )
cache_kwargs = {}
cache = StaticCacheLen(config, batch_size=36, max_cache_len=10, device="cpu", dtype=torch.bfloat16)
key_states = torch.randn(18, 16, 2, 128, dtype=torch.bfloat16, device="cpu")
value_states = torch.randn(18, 16, 2, 128, dtype=torch.bfloat16, device="cpu")

cache_position = torch.arange(key_states.shape[-2], device="cpu")
print(cache_position)
cache_kwargs["cache_position"] = cache_position
start_batch = 0
end_batch = 18
# copy to first
new_key_states, new_value_states=cache.update(key_states, value_states, 0, cache_kwargs, mode="update_batch", batch_dim=(start_batch, end_batch))
print(new_key_states.shape, new_value_states.shape)
print(cache.get_batch_length(0), cache.get_seq_length(0))

start_batch = cache.get_batch_length(0)
end_batch = start_batch + key_states.shape[0]
cache_position = torch.arange(key_states.shape[-2], device="cpu")
cache_kwargs["cache_position"] = cache_position
new_key_states, new_value_states=cache.update(key_states, value_states, 0, cache_kwargs, mode="update_batch", batch_dim=(start_batch, end_batch))
print(cache.get_batch_length(0), cache.get_seq_length(0))


key_states = torch.randn(36, 16, 1, 128, dtype=torch.bfloat16, device="cpu")
value_states = torch.randn(36, 16, 1, 128, dtype=torch.bfloat16, device="cpu")

seq_length = cache.get_seq_length(0)
cache_position = torch.arange(seq_length, seq_length+key_states.shape[-2], device="cpu")
print(cache_position)
cache_kwargs["cache_position"] = cache_position
new_key_states, new_value_states=cache.update(key_states, value_states, 0, cache_kwargs, mode="update_seq", batch_dim=(start_batch, end_batch))
print(cache.get_batch_length(0), cache.get_seq_length(0))




/root/miniconda3/envs/lpllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The 'batch_size' argument of StaticCacheLen is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of StaticCacheLen is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


tensor([0, 1])
torch.Size([36, 16, 10, 128]) torch.Size([36, 16, 10, 128])
18 2
36 2
tensor([2])
36 3


In [ ]:
import torch
import time
device="cuda:1"
weight_cache = torch.randn(1, 4096, 14336, dtype=torch.bfloat16, device="cpu").pin_memory()

In [ ]:
def scaled_dot_product_attention_with_pinned_memory(
    query: torch.Tensor,
    key: torch.Tensor, 
    value: torch.Tensor,
    output_tensor: Optional[torch.Tensor] = None,
    attn_mask: Optional[torch.Tensor] = None,
    dropout_p: float = 0.0,
    is_causal: bool = False,
    scale: Optional[float] = None,
    enable_gqa: bool = False
) -> torch.Tensor:
    """
    修改版的 scaled_dot_product_attention，支持将结果直接写入预分配的 pinned memory。
    
    Args:
        query: Query tensor of shape (..., L, E)
        key: Key tensor of shape (..., S, E) 
        value: Value tensor of shape (..., S, Ev)
        output_tensor: 预分配的 pinned memory tensor，用于存储结果。如果为 None，则创建新的 tensor
        attn_mask: 可选的注意力掩码
        dropout_p: Dropout 概率
        is_causal: 是否使用因果掩码
        scale: 缩放因子，如果为 None 则使用 1/sqrt(E)
        enable_gqa: 是否启用分组查询注意力
        
    Returns:
        注意力输出 tensor，如果提供了 output_tensor 则返回该 tensor
    """
    L, S = query.size(-2), key.size(-2)
    scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    
    # 计算输出形状
    output_shape = query.shape[:-1] + (value.shape[-1],)
    
    # 如果提供了预分配的输出 tensor，验证其形状和类型
    if output_tensor is not None:
        # 注意：对于 GPU tensor，即使原始是 pinned memory，移动到 GPU 后也不再是 pinned
        if output_tensor.shape != output_shape:
            raise ValueError(f"output_tensor 形状 {output_tensor.shape} 与期望形状 {output_shape} 不匹配")
        if output_tensor.dtype != query.dtype:
            raise ValueError(f"output_tensor 数据类型 {output_tensor.dtype} 与 query 数据类型 {query.dtype} 不匹配")
        if output_tensor.device != query.device:
            raise ValueError(f"output_tensor 设备 {output_tensor.device} 与 query 设备 {query.device} 不匹配")
    
    # 创建注意力偏置
    attn_bias = torch.zeros(L, S, dtype=query.dtype, device=query.device)
    
    if is_causal:
        assert attn_mask is None, "is_causal 和 attn_mask 不能同时使用"
        temp_mask = torch.ones(L, S, dtype=torch.bool, device=query.device).tril(diagonal=0)
        attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
        attn_bias = attn_bias.to(query.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask

    # 处理分组查询注意力
    if enable_gqa:
        key = key.repeat_interleave(query.size(-3)//key.size(-3), -3)
        value = value.repeat_interleave(query.size(-3)//value.size(-3), -3)

    # 计算注意力权重
    attn_weight = query @ key.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias
    attn_weight = torch.softmax(attn_weight, dim=-1)
    
    # 应用 dropout
    if dropout_p > 0.0:
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    
    # 计算最终输出
    if output_tensor is not None:
        # 直接写入预分配的 pinned memory
        torch.matmul(attn_weight, value, out=output_tensor)
        return output_tensor
    else:
        # 创建新的 tensor
        return attn_weight @ value

In [ ]:

# 960MB*3 = 2880MB
time_start = time.time()
for i in range(3*8):
    gpu_weight_cache = weight_cache.to(device="cuda:1")
print(f"time cost to cuda:1 {time.time() - time_start} s")

In [ ]:
import torch
import time
batch_size = 360
device="cuda:1"
query_states = torch.randn(batch_size, 32, 1, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
# 480MB, *4 = 1920MB
key_states = torch.randn(batch_size, 8, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
value_states = torch.randn(batch_size, 8, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
# 960MB*3 = 2880MB
time_start = time.time()
for i in range(4):
    key_states.to(device="cuda:1")
    value_states.to(device="cuda:1")
print(f"time cost to cuda:1 {time.time() - time_start} s")

In [19]:
import time
hidden_states_cache = torch.zeros(batch_size, 8, 4, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
# Copied from transformers.models.llama.modeling_llama.repeat_kv
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    # 改为将hidden_states 复制到 hidden_states_cache 上
    # for i in range(hidden_states_cache.shape[2]):
        # 这行代码的作用是：将 hidden_states 的内容复制到 hidden_states_cache 的第 i 个位置（第三个维度）。
        # 也就是说，对于 hidden_states_cache 的每个“副本”维度 i，都用同样的 hidden_states 填充，实现了在第3维上重复 hidden_states 的效果。
        # hidden_states_cache[:, :, i, :, :] = hidden_states
        # hidden_states_cache[:, :, i, :, :].index_copy_(3, torch.arange(hidden_states.shape[3]), hidden_states)
    hidden_states_cache = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    # hidden_states_cache = hidden_states.unsqueeze(2).expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states_cache.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)
def repeat_kv_optimized_v2(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    使用expand + reshape优化版本
    """
    if n_rep == 1:
        return hidden_states
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    # 在第1维上扩展n_rep倍
    expanded = hidden_states.unsqueeze(2).expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return expanded.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)
# attn_mask = torch.ones((480,512), dtype=torch.bfloat16, device="cpu")
print(f"key_states {key_states.device}")
time_start = time.time()
new_key_states = repeat_kv(key_states, 4)
new_value_states = repeat_kv(value_states, 4)
print(f"time cost repeat kv {time.time() - time_start} s")

time_start = time.time()
attn_output = torch.nn.functional.scaled_dot_product_attention(
    query_states,
    new_key_states,
    new_value_states,
    attn_mask=None,
    dropout_p=0.0,
    # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
    is_causal=False,
)
print(f"dot attn cost {time.time()-time_start:.6f} seconds")

key_states cpu
time cost repeat kv 0.7527637481689453 s
dot attn cost 0.071000 seconds


In [ ]:
# 测量移动整层layer需要的耗时
import torch
import time
batch_size = 360
device="cuda:1"
# hd = 16384
# hdd= 6144
hd = 14336
hdd= 4096
w0 = torch.randn(hd, hdd, dtype=torch.bfloat16, device="cpu").pin_memory()
w1 = torch.randn(hd, hdd, dtype=torch.bfloat16, device="cpu").pin_memory()
w2 = torch.randn(hd, hdd, dtype=torch.bfloat16, device="cpu").pin_memory()

time_start = time.time()
for i in range(8):
    w0_gpu = w0.to(device="cuda:1")
    w1_gpu = w1.to(device="cuda:1")
    w2_gpu = w2.to(device="cuda:1")
print(f"time cost to cuda:1 {time.time() - time_start} s")

In [ ]:
import torch
torch.cuda.empty_cache()

In [1]:
import torch
import time
# 1440MB
src = torch.randn(360, 32, 512, 128, dtype=torch.float16, device="cpu")
dst = torch.randn(360, 32, 512, 128, dtype=torch.float16, device="cpu")


In [2]:

key = torch.randn(360, 8, 512, 128, dtype=torch.float16, device="cpu")
key_dst = torch.randn(360, 8, 512, 128, dtype=torch.float16, device="cpu")

In [3]:

src_pin = src.pin_memory()
dst_pin = dst.pin_memory()

In [4]:
time_start = time.time()
for i in range(4):
    dst[:, i*8:i*8+8, :, :] = key
print(f"time cost pin copy {time.time()-time_start} s")

time_start = time.time()
for i in range(1):
    dst.copy_(src)
print(f"time cost {time.time()-time_start} s")

time_start = time.time()
for i in range(1):
    dst_pin.copy_(src_pin)
print(f"time cost pin copy {time.time()-time_start} s")

time_start = time.time()
for i in range(4):
    key_dst.copy_(key)
print(f"time cost pin copy {time.time()-time_start} s")

time cost pin copy 0.10234665870666504 s
time cost 0.07175183296203613 s
time cost pin copy 0.08554410934448242 s
time cost pin copy 0.06585931777954102 s


In [14]:
import torch
import time
from typing import Optional
import math
from torch.nn.attention import sdpa_kernel, SDPBackend
def scaled_dot_product_attention_with_pinned_memory(
    query: torch.Tensor,
    key: torch.Tensor, 
    value: torch.Tensor,
    output_tensor: Optional[torch.Tensor] = None,
    attn_mask: Optional[torch.Tensor] = None,
    dropout_p: float = 0.0,
    is_causal: bool = False,
    scale: Optional[float] = None,
    enable_gqa: bool = False
) -> torch.Tensor:
    """
    修改版的 scaled_dot_product_attention，支持将结果直接写入预分配的 pinned memory。
    
    Args:
        query: Query tensor of shape (..., L, E)
        key: Key tensor of shape (..., S, E) 
        value: Value tensor of shape (..., S, Ev)
        output_tensor: 预分配的 pinned memory tensor，用于存储结果。如果为 None，则创建新的 tensor
        attn_mask: 可选的注意力掩码
        dropout_p: Dropout 概率
        is_causal: 是否使用因果掩码
        scale: 缩放因子，如果为 None 则使用 1/sqrt(E)
        enable_gqa: 是否启用分组查询注意力
        
    Returns:
        注意力输出 tensor，如果提供了 output_tensor 则返回该 tensor
    """
    L, S = query.size(-2), key.size(-2)
    scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    
    # 计算输出形状
    output_shape = query.shape[:-1] + (value.shape[-1],)
    
    # 如果提供了预分配的输出 tensor，验证其形状和类型
    if output_tensor is not None:
        # 注意：对于 GPU tensor，即使原始是 pinned memory，移动到 GPU 后也不再是 pinned
        if output_tensor.shape != output_shape:
            raise ValueError(f"output_tensor 形状 {output_tensor.shape} 与期望形状 {output_shape} 不匹配")
        if output_tensor.dtype != query.dtype:
            raise ValueError(f"output_tensor 数据类型 {output_tensor.dtype} 与 query 数据类型 {query.dtype} 不匹配")
        if output_tensor.device != query.device:
            raise ValueError(f"output_tensor 设备 {output_tensor.device} 与 query 设备 {query.device} 不匹配")
    
    # 创建注意力偏置
    attn_bias = torch.zeros(L, S, dtype=query.dtype, device=query.device)
    
    if is_causal:
        assert attn_mask is None, "is_causal 和 attn_mask 不能同时使用"
        temp_mask = torch.ones(L, S, dtype=torch.bool, device=query.device).tril(diagonal=0)
        attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
        attn_bias = attn_bias.to(query.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask

    # 处理分组查询注意力
    if enable_gqa:
        key = key.repeat_interleave(query.size(-3)//key.size(-3), -3)
        value = value.repeat_interleave(query.size(-3)//value.size(-3), -3)

    # 计算注意力权重
    attn_weight = query @ key.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias
    attn_weight = torch.softmax(attn_weight, dim=-1)
    
    # 应用 dropout
    if dropout_p > 0.0:
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    
    # 计算最终输出
    if output_tensor is not None:
        # 直接写入预分配的 pinned memory
        torch.matmul(attn_weight, value, out=output_tensor)
        return output_tensor
    else:
        # 创建新的 tensor
        return attn_weight @ value

batch_size = 90
device="cuda:1"
query_states = torch.randn(batch_size, 32, 1, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
# 480MB, *4 = 1920MB
key_states = torch.randn(batch_size, 8, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
value_states = torch.randn(batch_size, 8, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()

hidden_states_cache = torch.zeros(batch_size, 8, 4, 512, 128, dtype=torch.bfloat16, device="cpu").pin_memory()
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1: 
        return hidden_states
    # 改为将hidden_states 复制到 hidden_states_cache 上
    # for i in range(hidden_states_cache.shape[2]):
        # 这行代码的作用是：将 hidden_states 的内容复制到 hidden_states_cache 的第 i 个位置（第三个维度）。
        # 也就是说，对于 hidden_states_cache 的每个“副本”维度 i，都用同样的 hidden_states 填充，实现了在第3维上重复 hidden_states 的效果。
        # hidden_states_cache[:, :, i, :, :].index_copy_(
        #     2, torch.arange(hidden_states.shape[2]), hidden_states
        # )
        # hidden_states_cache[:, :, i, :, :] = hidden_states
        # hidden_states_cache[:, :, i, :, :].index_copy_(3, torch.arange(hidden_states.shape[3]), hidden_states)
    hidden_states_cache = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    # hidden_states_cache = hidden_states.unsqueeze(2).expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states_cache.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)
print(f"key_states {key_states.device}")
time_start = time.time()
torch.cuda.nvtx.range_push("repeat kv")
new_key_states = repeat_kv(key_states, 4)
new_value_states = repeat_kv(value_states, 4)
torch.cuda.nvtx.range_pop()

# if torch.isnan(key_states).any():
#     print("NaN detected in key_states")
# if torch.isnan(value_states).any():
#     print("NaN detected in value_states")
# torch.cuda.synchronize()
print(f"time cost repeat kv {time.time() - time_start} s")

attn_output_c = torch.randn(batch_size, 32, 1,  128, dtype=torch.bfloat16, device="cpu")

# torch.cuda.nvtx.range_push("move same attn_output")
# time_start = time.time()
# attn_output_c = attn_output_c.pin_memory()
# ag = attn_output_c.to(device="cuda:1")
# print(f"move same attn_output time cost {time.time()-time_start} s")
# torch.cuda.nvtx.range_pop()


key_states cpu
time cost repeat kv 0.11017584800720215 s


In [16]:
# torch.cuda.nvtx.range_push("dot attn")
time_start = time.time()
# with sdpa_kernel([SDPBackend.CUDNN_ATTENTION, SDPBackend.MATH, SDPBackend.EFFICIENT_ATTENTION, SDPBackend.FLASH_ATTENTION]):
attn_output = torch.nn.functional.scaled_dot_product_attention(
    query_states,
    new_key_states,
    new_value_states,
    attn_mask=None,
    dropout_p=0.0,
    enable_gqa = False,
    # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
    is_causal=False,
)
# torch.cuda.synchronize()
print(f"dot attn cost {time.time()-time_start:.6f} seconds")
# print(attn_output.shape)
# torch.cuda.nvtx.range_push("clarify to pin")
# attn_output_pin = attn_output.pin_memory()
# torch.cuda.nvtx.range_pop()
# torch.cuda.nvtx.range_push("move")
# attn_output_gpu = attn_output_pin.to(device="cuda:1")
# torch.cuda.nvtx.range_pop()
# torch.cuda.nvtx.range_pop()
# print(f"dot attn and move cost {time.time()-time_start:.6f} seconds")

attn_output_c = torch.empty_like(query_states, device="cpu", dtype=torch.bfloat16, pin_memory=True)
torch.cuda.nvtx.range_push("dot attn pin")
def func_pin():
    time_start = time.time()
    torch.cuda.nvtx.range_push("scaled dot product")
    attn_output = scaled_dot_product_attention_with_pinned_memory(
        query_states,
        new_key_states,
        new_value_states,
        output_tensor=attn_output_c,
        attn_mask=None,
        dropout_p=0.0,
        # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
        is_causal=True,
    )
    print(attn_output.shape)
    attn_output.to(device="cuda:1")
    torch.cuda.nvtx.range_pop()
    print(f"dot attn pin cost {time.time()-time_start:.6f} seconds")
func_pin()
torch.cuda.nvtx.range_pop()



dot attn cost 0.005899 seconds
torch.Size([90, 32, 1, 128])
dot attn pin cost 0.006668 seconds


-2

In [ ]:
# from sllm_store.transformers import save_model

# # Load a model from HuggingFace model hub.
# import torch
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained('/mnt/zhengcf3/models/deepseek-moe-16b-base', torch_dtype=torch.float16, trust_remote_code=True)

# # Replace './models' with your local path.
# save_model(model, '/mnt/zhengcf3/models/models/deepseek-moe-16b-base')
import sys
import os



# Add the current directory to Python path
sys.path.insert(0, '/mnt/zhengcf3/lpllm')
from lpllm.lpllm import LPLLM
from lpllm.StaticCacheLen import StaticCacheLen
import torch
from transformers import AutoTokenizer, AutoConfig
from transformers.cache_utils import Cache, DynamicCache, StaticCache


import time
storage_path = "/mnt/zhengcf3/models/models"
tdevice = "cuda:1"

# model_name = "deepseek-moe-16b-base"
model_name = "Mixtral-8x7B"
model_path = f"/mnt/zhengcf3/models/models/{model_name}"


lp = LPLLM(model_name, tdevice, storage_path, pool_size=8, chunk_size_mb=2048)
config = lp.config

time_start = time.time()
lp.start_load_into_gpu(0)
lp.wait_load_into_gpu(0)
print(f"load into gpu cost {time.time() - time_start} s")